In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from boardlaw.main import *

buffer_length = 16 
batch_size = 64*1024
n_envs = 8*1024
buffer_inc = batch_size//n_envs

worlds = worldfunc(n_envs)
agent = agentfunc()
opt = torch.optim.Adam(agent.evaluator.parameters(), lr=1e-2, amsgrad=True)

sched = torch.optim.lr_scheduler.LambdaLR(opt, lambda e: min(e/1000, 1))

run = runs.new_run('pavlov-test')

In [ ]:
buffer = []
idxs = cycle(learning.batch_indices(buffer_length, n_envs, batch_size, worlds.device))


In [ ]:
while len(buffer) < buffer_length:
    decisions = agent(worlds, value=True)
    new_worlds, transition = worlds.step(decisions.actions)
    buffer.append(arrdict.arrdict(
        worlds=worlds,
        decisions=decisions,
        transitions=transition).detach())
    worlds = new_worlds
    log.info('actor stepped')

In [ ]:
with logs.to_run(run), stats.to_run(run):
    chunk = arrdict.stack(buffer)
    chunk_stats(chunk, buffer_inc)

In [ ]:
stats.review(rule='1s')